In [1]:
import pandas as pd 
import numpy as np
from helpers.read_eco_data import preprocess
import matplotlib.pyplot as plt

In [2]:
df = preprocess('../esg_data/esg_data.json')

In [3]:
df = df.loc[df['eco_score'] > 0]
df = df.reset_index()
df.drop(['index'], axis = 1, inplace=True)

In [4]:
stocks = pd.DataFrame()

for stock_name in df['ticker']:
    try: 
        temp = pd.read_json(f'../data/{stock_name}.json')
        new_df = pd.DataFrame({'date' : temp['t'], f'{stock_name}_price': temp['c']})
        new_df['date'] = pd.to_datetime(new_df['date'], unit='s')

        if stocks.empty: 
            stocks = new_df
        else: 
            stocks = pd.merge(stocks, new_df, how = 'outer', on='date')
    except Exception as e: 
        print(f"*** error reading {stock_name} : ", e)


*** error reading ADANIGR :  Expected object or value
*** error reading ADANIT :  Expected object or value
*** error reading ADYAF :  Expected object or value
*** error reading AHLU :  Expected object or value
*** error reading ALBK :  Expected object or value
*** error reading ANDHRABANK :  Expected object or value
*** error reading APCO :  Expected object or value
*** error reading AZRE :  Expected object or value
*** error reading CARE :  Expected object or value
*** error reading  :  Expected object or value
*** error reading CORPBANK :  Expected object or value
*** error reading CRAFTSMA :  Expected object or value
*** error reading DELHIVER :  Expected object or value
*** error reading DLTA :  Expected object or value
*** error reading EQUITAS :  Expected object or value
*** error reading EQUITASB :  Expected object or value
*** error reading EROS :  Expected object or value
*** error reading 1005Z :  Expected object or value
*** error reading FCON :  Expected object or value
***

In [5]:
from testers.stationarity import test_stationary

# for each stock in stocks check stationarity
stationary_stocks = []
for stock in stocks.columns[1:]:
    test_res = test_stationary(stocks[stock])
    if 0.05 > test_res.p > 0.01: 
        stationary_stocks.append(stock)

In [6]:
stocks

,date,3MINDIA_price,5PAISA_price,AWL_price,ATGL_price,ADANIPORTS_price,ADANIPOWER_price,AEGISCHEM_price,ALLCARGO_price,AMARAJABAT_price,...,TCIEXP_price,TEAMLEASE_price,TECHM_price,TECHNOE_price,TEJASNET_price,TEXRAIL_price,FEDERALBNK_price,KTKBANK_price,KARURVYSYA_price,THERMAX_price
0,2010-07-01,2757.50,NaN,NaN,NaN,145.93,126.15,22.76,64.23,90.93,...,NaN,NaN,184.08,NaN,NaN,NaN,31.62,118.73,50.48,745.65
1,2010-07-02,2763.60,NaN,NaN,NaN,142.99,125.25,22.46,64.23,91.48,...,NaN,NaN,183.15,NaN,NaN,NaN,33.00,116.75,52.06,746.85
2,2010-07-05,2816.05,NaN,NaN,NaN,146.61,126.40,23.64,65.12,90.65,...,NaN,NaN,188.41,NaN,NaN,NaN,33.08,119.07,56.24,752.25
3,2010-07-06,2776.25,NaN,NaN,NaN,144.13,125.50,23.85,65.08,90.75,...,NaN,NaN,190.38,NaN,NaN,NaN,34.48,124.38,56.31,765.30
4,2010-07-07,2779.30,NaN,NaN,NaN,145.18,124.05,23.84,64.74,94.38,...,NaN,NaN,189.10,NaN,NaN,NaN,35.00,120.14,56.98,754.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,2023-08-16,28746.55,449.40,370.10,635.55,776.10,279.90,358.85,277.15,616.50,...,1425.45,2253.10,1227.85,476.35,843.00,118.80,132.20,225.00,119.05,2585.60
3256,2023-08-17,29174.65,442.80,368.10,633.70,809.65,286.55,352.85,273.55,622.30,...,1422.80,2238.65,1231.75,493.00,853.15,118.35,134.60,228.70,119.80,2579.10
3257,2023-08-18,29126.70,431.25,381.95,654.50,835.90,304.60,355.30,265.35,627.65,...,1413.85,2192.70,1210.80,483.15,843.90,113.95,134.15,227.65,116.85,2599.50
3258,2023-08-21,29034.20,436.55,382.50,664.50,858.15,324.85,349.95,268.50,627.80,...,1402.90,2212.85,1218.85,473.60,838.20,117.85,134.25,228.90,120.75,2766.50


In [7]:
def plot1(data, APO_VALUE_FOR_BUY_ENTRY = -10, APO_VALUE_FOR_SELL_ENTRY = 10): 
    data['ClosePrice'].plot(color='blue', lw=3., legend=True)
    data['Fast10DayEMA'].plot(color='y', lw=1., legend=True)
    data['Slow40DayEMA'].plot(color='m', lw=1., legend=True)
    plt.plot(data.loc[ data.Trades == 1 ].index,
    data.ClosePrice[data.Trades == 1 ], color='r', lw=0, marker='^',
    markersize=7, label='buy')
    plt.plot(data.loc[ data.Trades == -1 ].index,
    data.ClosePrice[data.Trades == -1 ], color='g', lw=0, marker='v',
    markersize=7, label='sell')
    plt.legend()
    plt.show()
    
    data['APO'].plot(color='k', lw=3., legend=True)
    plt.plot(data.loc[ data.Trades == 1 ].index, data.APO[data.Trades
    == 1 ], color='r', lw=0, marker='^', markersize=7, label='buy')
    plt.plot(data.loc[ data.Trades == -1 ].index, data.APO[data.Trades
    == -1 ], color='g', lw=0, marker='v', markersize=7, label='sell')
    plt.axhline(y=0, lw=0.5, color='k')
    
    for i in range( APO_VALUE_FOR_BUY_ENTRY, APO_VALUE_FOR_BUY_ENTRY*5,
    APO_VALUE_FOR_BUY_ENTRY ):
        plt.axhline(y=i, lw=0.5, color='r')
    for i in range( APO_VALUE_FOR_SELL_ENTRY,
    APO_VALUE_FOR_SELL_ENTRY*5, APO_VALUE_FOR_SELL_ENTRY ):
        plt.axhline(y=i, lw=0.5, color='g')
        
    plt.legend()
    plt.show()
 

In [8]:
def plot2(data, NUM_SHARES_PER_TRADE= 10): 
    data['Position'].plot(color='k', lw=1., legend=True)
    plt.plot(data.loc[ data.Position == 0 ].index, data.Position[
    data.Position == 0 ], color='k', lw=0, marker='.', label='flat')
    plt.plot(data.loc[ data.Position > 0 ].index, data.Position[
    data.Position > 0 ], color='r', lw=0, marker='+', label='long')
    plt.plot(data.loc[ data.Position < 0 ].index, data.Position[
    data.Position < 0 ], color='g', lw=0, marker='_', label='short')
    plt.axhline(y=0, lw=0.5, color='k')
    
    for i in range( NUM_SHARES_PER_TRADE, NUM_SHARES_PER_TRADE*25,
    NUM_SHARES_PER_TRADE*5 ):
        plt.axhline(y=i, lw=0.5, color='r')
        
    for i in range( -NUM_SHARES_PER_TRADE, -NUM_SHARES_PER_TRADE*25, -
    NUM_SHARES_PER_TRADE*5 ):
        plt.axhline(y=i, lw=0.5, color='g')
        
    plt.legend()
    plt.show()
    data['Pnl'].plot(color='k', lw=1., legend=True)
    plt.plot(data.loc[ data.Pnl > 0 ].index, data.Pnl[ data.Pnl > 0 ],
    color='g', lw=0, marker='.')
    plt.plot(data.loc[ data.Pnl < 0 ].index, data.Pnl[ data.Pnl < 0 ],
    color='r', lw=0, marker='.')
    plt.legend()
    plt.show()

In [9]:
def apo_based_trading(data): 
    # Variables/constants for EMA Calculation:
    NUM_PERIODS_FAST = 10 # Static time period parameter for the fast EMA
    K_FAST = 2 / (NUM_PERIODS_FAST + 1) 
    ema_fast = 0.0
    ema_fast_values = [] 
    
    NUM_PERIODS_SLOW = 40 # Static time period parameter for slow EMA
    K_SLOW = 2 / (NUM_PERIODS_SLOW + 1) 
    ema_slow = 0.0
    ema_slow_values = [] 
    apo_values = []
    
    orders, positions, pnls = [], [], []
    last_buy_price = 0 
    last_sell_price = 0 
    position = 0 
    buy_sum_price_qty = 0
    buy_sum_qty = 0 
    sell_sum_price_qty = 0 
    sell_sum_qty = 0 
    open_pnl = 0 
    closed_pnl = 0 
    
    APO_VALUE_FOR_BUY_ENTRY = -10 
    APO_VALUE_FOR_SELL_ENTRY = 10 
    MIN_PRICE_MOVE_FROM_LAST_TRADE = 10 
    MIN_PROFIT_TO_CLOSE = 10 
    NUM_SHARES_PER_TRADE = 10
    
    close = data['Price']
    for close_price in close:
        if ema_fast == 0: 
            ema_fast = close_price
            ema_slow = close_price
        else:
            ema_fast = (close_price - ema_fast) * K_FAST + ema_fast
            ema_slow = (close_price - ema_slow) * K_SLOW + ema_slow
        ema_fast_values.append(ema_fast)
        ema_slow_values.append(ema_slow)
        apo = ema_fast - ema_slow
        apo_values.append(apo)
        
        if ((apo > APO_VALUE_FOR_SELL_ENTRY and abs(close_price -
        last_sell_price) > MIN_PRICE_MOVE_FROM_LAST_TRADE)
        or
        (position > 0 and (apo >= 0 or open_pnl >
        MIN_PROFIT_TO_CLOSE))): 
            orders.append(-1) # mark the sell trade
            last_sell_price = close_price
            position -= NUM_SHARES_PER_TRADE # reduce position by the size of the trade
            sell_sum_price_qty += (close_price*NUM_SHARES_PER_TRADE) # update vwap sell-price
            sell_sum_qty += NUM_SHARES_PER_TRADE

        elif ((apo < APO_VALUE_FOR_BUY_ENTRY and abs(close_price -
        last_buy_price) > MIN_PRICE_MOVE_FROM_LAST_TRADE) 
        or
        (position < 0 and (apo <= 0 or open_pnl >
        MIN_PROFIT_TO_CLOSE))):
            orders.append(+1) 
            last_buy_price = close_price
            position += NUM_SHARES_PER_TRADE 
            buy_sum_price_qty += (close_price*NUM_SHARES_PER_TRADE) 
            buy_sum_qty += NUM_SHARES_PER_TRADE
        else:
            orders.append(0)
            
        positions.append(position)
        open_pnl = 0
        if position > 0:
            if sell_sum_qty > 0: 
                open_pnl = abs(sell_sum_qty) * (sell_sum_price_qty/sell_sum_qty - buy_sum_price_qty/buy_sum_qty)
                
                open_pnl += abs(sell_sum_qty - position) * (close_price -
                buy_sum_price_qty / buy_sum_qty)
        elif position < 0:
            if buy_sum_qty > 0:
                open_pnl = abs(buy_sum_qty) * (sell_sum_price_qty/sell_sum_qty - buy_sum_price_qty/buy_sum_qty)
                open_pnl += abs(buy_sum_qty - position) * (sell_sum_price_qty/sell_sum_qty - close_price)
        else:
            closed_pnl += (sell_sum_price_qty - buy_sum_price_qty)
            buy_sum_price_qty = 0
            buy_sum_qty = 0
            sell_sum_price_qty = 0
            sell_sum_qty = 0
            last_buy_price = 0
            last_sell_price = 0
        pnls.append(closed_pnl + open_pnl)
        
    data = data.assign(ClosePrice=pd.Series(close, index=data.index))
    data = data.assign(Fast10DayEMA=pd.Series(ema_fast_values,
    index=data.index))
    data = data.assign(Slow40DayEMA=pd.Series(ema_slow_values,
    index=data.index))
    data = data.assign(APO=pd.Series(apo_values, index=data.index))
    data = data.assign(Trades=pd.Series(orders, index=data.index))
    data = data.assign(Position=pd.Series(positions, index=data.index))
    data = data.assign(Pnl=pd.Series(pnls, index=data.index))

    return data

In [16]:
def analyze(new_df): 
    try: 
        # calculate drawdown
        new_df['max'] = new_df['Pnl'].cummax()
        new_df['drawdown'] = new_df['max'] - new_df['Pnl']
        plt.show()
        
        
        if new_df['Pnl'].mean() == 0.0:
            sharpe_ratio = 0
        else:
            sharpe_ratio = new_df['Pnl'].mean() / new_df['Pnl'].std()
        # calculate max drawdown
        max_drawdown = new_df['drawdown'].max()
        # # calculate max drawdown duration
        # max_drawdown_duration = new_df['drawdown'].idxmax() - new_df['drawdown'].idxmin()
        # calculate total return
        total_return = new_df['Pnl'].iloc[-1]
        # # calculate annualized return
        # annualized_return = new_df['Pnl'].iloc[-1] / len(new_df['Pnl']) * 252
        # # calculate annualized volatility
        # annualized_volatility = new_df['Pnl'].std() * np.sqrt(252)

        # calculate return percentage by dividing total reu
        # # calculate sortino ratio
        # sortino_ratio = new_df['Pnl'].mean() / new_df[new_df['Pnl'] < 0]['Pnl'].std()
        # # calculate calmar ratio
        # calmar_ratio = new_df['Pnl'].mean() / max_drawdown
        # # calculate average return
        # average_return = new_df['Pnl'].mean()
        # # calculate average loss
        # average_loss = new_df[new_df['Pnl'] < 0]['Pnl'].mean()
        # # calculate average win
        # average_win = new_df[new_df['Pnl'] > 0]['Pnl'].mean()
        # # calculate win rate
        # win_rate = len(new_df[new_df['Pnl'] > 0]['Pnl']) / len(new_df['Pnl'])
        # # calculate profit factor
        # profit_factor = new_df[new_df['Pnl'] > 0]['Pnl'].sum() / abs(new_df[new_df['Pnl'] < 0]['Pnl'].sum())
        # # create a dictionary using all the above variables that we have calculated
        
        
        # results = pd.Series({'sharpe_ratio': [sharpe_ratio], 'max_drawdown': [max_drawdown], 'max_drawdown_duration': [max_drawdown_duration], 'total_return': [total_return], 'annualized_return': [annualized_return], 'annualized_volatility': [annualized_volatility], 'sortino_ratio': [sortino_ratio], 'calmar_ratio': [calmar_ratio], 'average_return': [average_return], 'average_loss': [average_loss], 'average_win': [average_win], 'win_rate': [win_rate], 'profit_factor': [profit_factor]})
        results = pd.Series({'sharpe_ratio': [sharpe_ratio], 'max_drawdown': [max_drawdown], 'total_return': [total_return]})
       #  remove [] brackets from df
        results = results.apply(lambda x: x[0])
        return results
    
    except Exception as e: 
        print(e)
        

    

In [17]:
for stock in stationary_stocks:
    new_df = pd.DataFrame({'Price': stocks[f'{stock}']})
    new_df = apo_based_trading(new_df)
    
    try: 
        res = analyze(new_df)
        if res.sharpe_ratio != 0:
            print(f'{stock}\n', res)
    except Exception as e: 
        print(f"*** error analyzing {stock} : ", e)

ANANTRAJ_price
 sharpe_ratio       0.016997
max_drawdown     393.875000
total_return    6972.800000
dtype: float64
CCL_price
 sharpe_ratio         0.348679
max_drawdown     97978.076923
total_return    310422.861111
dtype: float64
HDIL_price
 sharpe_ratio      -2.226567
max_drawdown    5374.350000
total_return    -859.500000
dtype: float64
ITC_price
 sharpe_ratio        0.963431
max_drawdown     7036.933333
total_return    35449.833333
dtype: float64
MMTC_price
 sharpe_ratio        -1.969870
max_drawdown    272769.363523
total_return   -117556.300000
dtype: float64
NTPC_price
 sharpe_ratio      0.829557
max_drawdown      0.000000
total_return    609.700000
dtype: float64
RELINFRA_price
 sharpe_ratio         0.798260
max_drawdown    211339.833333
total_return     74417.500000
dtype: float64
SHRIRAMFIN_price
 sharpe_ratio    1.260891e+00
max_drawdown    5.893353e+05
total_return    1.135294e+06
dtype: float64
KTKBANK_price
 sharpe_ratio       1.994742
max_drawdown    1314.371429
total_re